In [1]:
from modules import *
from sim import *
from read_ramses import *

import yt


In [2]:
# choose simulation
sim_idx = GAS

sim_name = list_of_sim_name[sim_idx]
sim_latex = list_of_sim_latex[sim_idx]
sim_dir = os.path.join(sim_base_dir, sim_name)

os.chdir(sim_dir)


In [3]:
halo = get_biggest_halos(1, use_density=False)
info_file = get_stdout("ls output*/info*").split()[0]

halo.coord = np.array([5.105151466E-01, 5.127534424E-01, 4.929928780E-01])
    

In [4]:
ds = yt.load(info_file)


yt : [INFO     ] 2023-10-06 16:43:06,466 Parameters: current_time              = 4.006925024948247
yt : [INFO     ] 2023-10-06 16:43:06,467 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2023-10-06 16:43:06,468 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-10-06 16:43:06,468 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2023-10-06 16:43:06,468 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-10-06 16:43:06,469 Parameters: current_redshift          = 8.999851699314442
yt : [INFO     ] 2023-10-06 16:43:06,469 Parameters: omega_lambda              = 0.723999977111816
yt : [INFO     ] 2023-10-06 16:43:06,469 Parameters: omega_matter              = 0.275999993085861
yt : [INFO     ] 2023-10-06 16:43:06,469 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-10-06 16:43:06,469 Parameters: hubble_constant           = 0.703000030517578


In [5]:
# read constants from yt

a_exp = ds["aexp"]
H0 = ds["H0"] * const.km / const.Mpc
Omega_m0 = ds["omega_m"]
Omega_L0 = ds["omega_l"]
Omega_k0 = ds["omega_k"]
Omega_b0 = ds["omega_b"]

length_unit = ds["unit_l"]
density_unit = ds["unit_d"]
time_unit = ds["unit_t"]

mass_unit = density_unit * length_unit**3
vel_unit = length_unit / time_unit
energy_unit = mass_unit * vel_unit**2
energy_density_unit = density_unit * vel_unit**2

halo.convert_to_cgs(length_unit, density_unit)


In [6]:
# define size and resolution of the region of interest

max_amr_level = 13
box_size = 2 * const.kpc

left_edge = halo.coord - box_size / 2
N = int(box_size / length_unit * ds.domain_dimensions[0] * 2**max_amr_level)


In [7]:
# get raw yt data from the region of interest

data = ds.covering_grid(level=max_amr_level, left_edge=(left_edge / length_unit), dims=[N]*3)


In [8]:
# read particle data from yt

particle_type = data["io", "particle_family"].value.astype(int)
is_dm = particle_type == DM
is_star = particle_type == STAR

coord_dm = (np.array([data["io", "particle_position_x"].value[is_dm], data["io", "particle_position_y"].value[is_dm], data["io", "particle_position_z"].value[is_dm]])) * length_unit - left_edge[:, None] - box_size / 2
mass_dm = data["io", "particle_mass"][is_dm] * mass_unit

coord_star = (np.array([data["io", "particle_position_x"].value[is_star], data["io", "particle_position_y"].value[is_star], data["io", "particle_position_z"].value[is_star]])) * length_unit - left_edge[:, None] - box_size / 2
mass_star = data["io", "particle_mass"][is_star] * mass_unit
tau_starbirth = data["io", "conformal_birth_time"][is_star].value


yt : [INFO     ] 2023-10-06 16:43:28,543 Adding particle_type: DM
yt : [INFO     ] 2023-10-06 16:43:28,553 Adding particle_type: star
yt : [INFO     ] 2023-10-06 16:43:28,563 Adding particle_type: cloud
yt : [INFO     ] 2023-10-06 16:43:28,572 Adding particle_type: dust
yt : [INFO     ] 2023-10-06 16:43:28,582 Adding particle_type: star_tracer
yt : [INFO     ] 2023-10-06 16:43:28,591 Adding particle_type: cloud_tracer
yt : [INFO     ] 2023-10-06 16:43:28,601 Adding particle_type: dust_tracer
yt : [INFO     ] 2023-10-06 16:43:28,610 Adding particle_type: gas_tracer


In [11]:
# read gas data from yt

coord = (np.array([data["index", "x"].value, data["index", "y"].value, data["index", "z"].value])) * length_unit - left_edge[:, None, None, None] - box_size / 2
density = data["ramses", "Density"].value * density_unit
metallicity = data["ramses", "Metallicity"].value 
pressure = data["ramses", "Pressure"].value * energy_density_unit
energy_turb = data["ramses", "hydro_scalar_01"].value * vel_unit**2
ref_crit = data["ramses", "hydro_scalar_02"].value
vel_vec = np.array([data["ramses", "x-velocity"].value, data["ramses", "y-velocity"].value, data["ramses", "z-velocity"].value]) * vel_unit


In [22]:
safe_savez(
    "gridded_data_small",
    halo_idx=halo.idx,
    halo_mass=halo.mass,
    a_exp=a_exp,
    H0=H0,
    Omega_m0=Omega_m0,
    Omega_L0=Omega_L0,
    Omega_k0=Omega_k0,
    Omega_b0=Omega_b0,
    length_unit=length_unit,
    density_unit=density_unit,
    time_unit=time_unit,
    mass_unit=mass_unit,
    vel_unit=vel_unit,
    energy_unit=energy_unit,
    energy_density_unit=energy_density_unit,
    max_amr_level=max_amr_level,
    box_size=box_size,
    left_edge=left_edge,
    N=N,
    coord_dm=coord_dm,
    mass_dm=mass_dm,
    coord_star=coord_star,
    mass_star=mass_star,
    tau_starbirth=tau_starbirth,
    coord=coord,
    density=density,
    metallicity=metallicity,
    pressure=pressure,
    energy_turb=energy_turb,
    ref_crit=ref_crit,
    vel_vec=vel_vec
)


File 'gridded_data_small' already exists. Would you like to overwrite? [yes/no]: yes
Overwriting file.
